In [1]:
import torch
from torch import autograd
import numpy as np

In [2]:
device = torch.device('cpu')
if torch.cuda.is_available():
    print("CUDA is available! Training on GPU.")
    device = torch.device('cuda:0')
options_0 = {'dtype': torch.float64, 'device': device, 'requires_grad': True}
options_1 = {'dtype': torch.float64, 'device': device, 'requires_grad': False}

s0 = 100; k = 100; r = 0.02; q = 0.01; t = 0.25; sigma = 0.15;
m = 50000 # num of sim
typ = 1
# t = 1
s0 = 90
ntimes= int(t*12*5)
s_0 = torch.tensor(s0, **options_0)
dt = t / ntimes
ts = torch.arange(0.0, t+dt, dt, **options_1)
e = torch.randn(m, ntimes, **options_1)
sum_e = torch.cat((torch.zeros(m,1,**options_1), torch.cumsum(e,1)),1)
s = s_0 * torch.exp((r-q-0.5*sigma*sigma) * ts + sigma * np.sqrt(dt) * sum_e)
# s = s_0 * s_0 * torch.exp((r-q-0.5*sigma*sigma) * ts + sigma * np.sqrt(dt) * sum_e)
disc = np.exp(-r*t)
sAtMat = s[:,-1]
discPayoff = disc * torch.relu(typ *( sAtMat - k));
# discPayoff = disc * torch.max(typ *( sAtMat - k), torch.tensor(0.0));
price = discPayoff.mean();
first_derivative = autograd.grad(price, s_0, create_graph=True)[0]
# first_derivative.requires_grad = True
# We now have dloss/dx
second_derivative = autograd.grad(first_derivative, s_0, allow_unused=True)
# price.backward();
# delta = s_0.grad.item()
# The second_derivative for gamma does not work since the payoff function is only C0 continuous

In [3]:
price

tensor(0.2686, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [4]:
first_derivative

tensor(0.0886, dtype=torch.float64, grad_fn=<SumBackward0>)

In [5]:
second_derivative

(tensor(0., dtype=torch.float64),)

# Hessian

In [31]:
def bsm_price(x):
    s_0 = x
    dt = t / ntimes
    ts = torch.arange(0.0, t+dt, dt, **options_1)
    e = torch.randn(m, ntimes, **options_1)
    sum_e = torch.cat((torch.zeros(m,1,**options_1), torch.cumsum(e,1)),1)
    s = s_0 * torch.exp((r-q-0.5*sigma*sigma) * ts + sigma * np.sqrt(dt) * sum_e)
    disc = np.exp(-r*t)
    sAtMat = s[:,-1]
    discPayoff = disc * torch.relu(typ *( sAtMat - k));
    price = discPayoff.mean();
    return price

def bsm_price_soft(x, beta=1):
    s_0 = x
    dt = t / ntimes
    ts = torch.arange(0.0, t+dt, dt, **options_1)
    e = torch.randn(m, ntimes, **options_1)
    sum_e = torch.cat((torch.zeros(m,1,**options_1), torch.cumsum(e,1)),1)
    s = s_0 * torch.exp((r-q-0.5*sigma*sigma) * ts + sigma * np.sqrt(dt) * sum_e)
    disc = np.exp(-r*t)
    sAtMat = s[:,-1]
    discPayoff = disc * torch.nn.Softplus(beta=beta)(typ *( sAtMat - k)); # It works!
    price = discPayoff.mean();
    return price

In [32]:
bsm_price(s_0)

tensor(0.2815, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [33]:
torch.autograd.functional.jacobian(bsm_price, s_0)

tensor(0.0905, dtype=torch.float64)

In [34]:
torch.autograd.functional.hessian(bsm_price, s_0)

tensor(0., dtype=torch.float64)

In [43]:
bsm_price_soft(s_0,20.0)
soft_price = lambda x : bsm_price_soft(x, 20.0)

In [44]:
torch.autograd.functional.jacobian(soft_price, s_0)

tensor(0.0918, dtype=torch.float64)

In [45]:
torch.autograd.functional.hessian(soft_price, s_0)

tensor(0.0243, dtype=torch.float64)

In [ ]:
	double s = 100, k = 100, r = 0.02, q = 0.01, t = 0.25, sigma = 0.15;
	OptionType type = Call;
	unsigned int ntimes = 100, numOfSimulation = 50000;
	torch::Tensor s_0 = torch::tensor(s0, options_0);

	double dt = t / ntimes;
	//int ntimes = int(t / dt);
	torch::Tensor ts = torch::arange(0.0, t + dt, dt, options_1); // first elem:0, last elem: t

	torch::Tensor e = torch::randn({ m, ntimes }, options_1);
	torch::Tensor sum_e = torch::cumsum(e, 1); // [0-th e[0], e[0]+e[1], ..., e[0]+e[n]+...e[m-1]; 1-th ...;...]
	sum_e = torch::cat({ torch::zeros({ m, 1 }, options_1), sum_e }, 1); // concatenate a col of zeros + sum_e
	torch::Tensor s = s_0 * torch::exp((r - q - 0.5 * sigma * sigma) * ts + sigma * sqrt(dt) * sum_e);

	double disc = exp(-r * t);
	auto sAtMat = s.index({ torch::arange(0, s.size(0), torch::kLong), torch::tensor(s.size(1) - 1) });
	torch::Tensor discPayoff = disc * torch::relu(type *( sAtMat - k));
	auto price = discPayoff.mean();
	price.backward();
	// return PriceResult({ price.item<double>(), s_0.grad().item<double>() });
	return PriceResult({ price.item<double>(), s_0.grad().item<double>(), s_0.grad().grad().item<double>() });

In [ ]:
sqrt